In [56]:
from elasticsearch import Elasticsearch
import json
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search,Q
from credentials import usr_pwd

In [10]:
hosts = ["10.1.8.6:9200"]
user,pas = usr_pwd()

In [11]:
es = Elasticsearch(hosts,
                  timeout = 6000,
                  use_ssl = True,
                  verify_certs = False,
                  ssl_show_warn = False,
                  raise_on_error = False,
                  http_auth = (user,pas)
                  )

In [12]:
es.ping()

True

In [5]:
body = {
    "size":1000,
    "query":{
        "bool":{
#             "filter":{
#                 "exists":{
#                     "field":"Attachment"
#                 }
#             },
            "must":[
                {
                "match_phrase":{
                    "content":"rmb devaluation"
                }
            }
#             ,{
#                 "match_phrase_prefix":{
#                     "fileType":"pdf"
#                 }
#             }
            ]
        }
    }
}
res = es.search(index=["*"],body=body)
print(len(res["hits"]["hits"]))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


996


In [ ]:
# res["hits"]["hits"][0]

## Extracting metadata based on phrase

In [13]:
body = {
    "size":5000,
    "query":{
        "bool":{
            "must":[{
                "match_phrase_prefix":{
                    "content":"For 97 years, Canadian Tire Retail"
#                     "content":"For 97 years, Canadian Tire Retail has maintained a strong relationship with its customers from coast-to-coast."
#                     "content":"For 97 years, Canadian Tire Retail has maintained a strong relationship with its customers from coast-to-coast.  As the retail market evolves and competition increases, we have been investing to build improved customer acquisition capabilities within our organization. These developing capabilities have played a key role in delivering 21 consecutive quarters of growth at CTR.  Our success is due in large part to incremental investments we are making to aggressively market new customer-centric vehicles that enable us to connect with and reward customers in new ways and build loyalty to drive sustainable growth. At the same time, we are investing in our traditional vehicles that continue to deliver strong results. As our relevance with our customers has grown, so has our collective business."
                }
            },
            {
                "match_phrase_prefix":{
                    "fileType":"Email"
                }
            }]
        }
    }
}

In [15]:
res = es.search(index=["cantire-2020-*","cantire-2021-*"],body=body)
print(len(res["hits"]["hits"]))

#2222 - match_phrase_prefix for cantire-202*
#878 - cantire-2020*,cantire-2021* - full phrase
#2035 - first sentence with above index.
#2493 - for short phrase ("For 97 years, Canadian Tire Retail")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.


2493


In [61]:
res['hits']["hits"][2]["_source"]
# thread_id = []
# for i in range(100):
#     try:
#         thread_id.append(res["hits"]["hits"][i]["_source"]["metaData"]["thread-index"])
#     except:
#         print(i)
# 20200904082345115.2B286ABC4C645959@XS203


{'content': 'Hi Tina,\r\n\r\nWe received the approval information from your vendor support team about the change. Thank you.\r\n\r\n\r\n\r\nBest regards,\r\nHayley Xu \r\n\r\n\r\n\r\nShanghai Greatway Industry Co.,Ltd\r\nTel:  +86-021-64501185 ext 6532\r\nFax: +86-021-64508336\r\nMobile:+86-18721829914\r\nWeb: www.guangwei-china.com \r\n \r\n----- Original Message -----\r\nFrom：Hayley Xu\r\nDate：2020-09-04 16:09\r\nSubject：Re: RE: RE: RE: Re: 1% Vendor Fund Initiative - CT investment and Vendor support\r\nTo：Tina Zhang\r\n\r\nHi Tina,\r\n\r\nThanks for your information.\r\n\r\nThe terms is revised and resubmitted. Please help to check it , thank you.\r\n\r\n\r\nBest regards,\r\nHayley Xu \r\n\r\n\r\n\r\nShanghai Greatway Industry Co.,Ltd\r\nTel:  +86-021-64501185 ext 6532\r\nFax: +86-021-64508336\r\nMobile:+86-18721829914\r\nWeb: www.guangwei-china.com \r\n  \r\n----- Original Message -----\r\nFrom：Tina Zhang\r\nDate：2020-09-04 09:23\r\nSubject：RE: RE: RE: Re: 1% Vendor Fund Initiative

In [7]:
# for i in range(len(res["hits"]["hits"])):
#     if "For 97 years, Canadian Tire Retail" not in res["hits"]["hits"][i]["_source"]["content"]:
#         print(i)

In [16]:
def domain_ext(val,i):
    value = []
    try:
        k = res["hits"]["hits"][i]["_source"]["metaData"][val]
    except:
        k=""
    m = []
    if type(k)!=str:
        m.extend(k)
        if len(k)<=1:
            s = (" ".join(m)).split("@")
            value.append(s[1])
        else:
            for j in k:
                if "@" in j:
                    s = ("".join(j)).split("@")
                    value.append(s[1])
    else:
        m.append(k)
        s = (" ".join(m)).split("@")
        try:
            value.append(s[1])
        except:
            pass
    return list(set(value))

In [17]:
import re
def date_ext(dt,i):
    cont = res["hits"]["hits"][i]["_source"]["content"]
    cont = cont.split("From:")
    
    if len(cont)>1:
        day = cont[dt].find("Sent:")
        if day == -1:
            day = (cont[dt].find("Date:"))
            if day !=-1:
                day+=5
                resu = cont[dt][day:day+30]
                resu = resu.replace(","," ").replace("."," ").replace("\n","").replace("*","")
                resu_r = resu.find("\r")
                if resu_r !=-1:
                    resu = resu[:resu_r]
                    
                resu_t = resu.find("To")
                if resu_t !=-1:
                    resu = resu[:resu_t]
                try:
                    x = re.search("\s\d{4}\s", resu)
                    result = resu[:x.end()]
                    return result
                except:
                    return resu

        else:
            day = day+5
            resu = cont[dt][day:day+30]
            resu = resu.replace(","," ").replace("."," ").replace("\n","").replace("*","")
            resu_r = resu.find("\r")
            if resu_r !=-1:
                resu = resu[:resu_r]
                
            resu_t = resu.find("To")
            if resu_t !=-1:
                resu = resu[:resu_t]
            try:
                x = re.search("\s\d{4}\s", resu)
                result = resu[:x.end()]
                return result
            except:
                return resu
    else:
        return res["hits"]["hits"][i]["_source"]["metaData"]["date"]

In [18]:
pwd

'/mnt/cantire-analytics/datarepo/pavan rs/elasticsearch_works'

In [21]:
cd zz

/mnt/cantire-analytics/datarepo/zz


In [218]:
# /mnt/cantire-analytics/datarepo/zz     the below created csv is in this path

In [22]:
import csv
header = ["md5","subject","from_domain","to_domain","begin_email_date","end_email_date","Domains"]
with open("md_97y_phrase_1.csv", "a") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(header)

In [23]:
import csv
from tqdm import tqdm
j=0
lis = []
for i in range(len(res["hits"]["hits"])):
    j+=1
    md5 = []
    sub = []
    frm = []
    to = []
    b_date = []
    e_date = []
    
    try:
        m = res["hits"]["hits"][i]["_source"]["MD5"]
        if type(m)!=str:
            md5.extend(m)
        else:
            md5.append(m)
    except:
        pass
    
    try:
        s = res["hits"]["hits"][i]["_source"]["metaData"]["subject"]
        if type(s)!=str:
            sub.extend(s)
        else:
            sub.append(s)
    except:
        pass
    
    frm.extend(domain_ext("from",i))
    to.extend(domain_ext("to",i))
    b_date.append(date_ext(-1,i))     # -1 because the latest mail will be the first mail in the thread
    e_date.append(date_ext(1,i))    # 1 becasue the initial mail of the thread will be at last of the thread.
    dom = frm + to
    for I in dom:
        if I.lower() == "cantire.com":
            dom.remove(I)
        if len(dom) == 0:
            dom.append("cantire.com")
    dom = ",".join(dom)
#     print(md5,sub,frm,to,b_date,e_date,dom)

    lis.extend([[md5,sub,frm,to,b_date,e_date,dom]])
    if i==100:
        with open("md_97y_phrase_1.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(lis)
        lis=[]
        i=0
if i!=0:
    with open("md_97y_phrase_1.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(lis)

In [41]:
# import re
# for i in range(100):
#     cont = res["hits"]["hits"][i]["_source"]["content"]
#     cont = cont.split("From:")
    
#     if len(cont)>1:
#         day = cont[-1].find("Sent:")
#         if day == -1:
#             day = (cont[-1].find("Date:"))
#             if day !=-1:
#                 day+=5
#                 resu = cont[-1][day:day+30]
#                 resu = resu.replace(","," ").replace("."," ").replace("\n","")
#                 try:
#                     x = re.search("\s\d{4}\s", resu)
#                     result = resu[:x.end()]
#                     print(i,result)
#                 except:
#                     print(i,resu)

#         else:
#             day = day+5
#             resu = cont[-1][day:day+30]
#             resu = resu.replace(","," ").replace("."," ").replace("\n","").replace("*","")
#             try:
#                 x = re.search("\s\d{4}\s", resu)
#                 result = resu[:x.end()]
#                 print(result)
#             except:
#                 print(resu)
#     else:
#         print(res["hits"]["hits"][i]["_source"]["metaData"]["date"])
        

In [ ]:

# for i in range(len(res["hits"]["hits"])):
#     val="to"
#     value = []
#     try:
#         k = res["hits"]["hits"][i]["_source"]["metaData"][val]
#     except:
#         k=""
#     m = []
#     if type(k)!=str:
#         m.extend(k)
#         if len(k)<=1:
#             s = (" ".join(m)).split("@")
#             value.append(s[1])
#         else:
#             for j in k:
#                 if "@" in j:
#                     s = ("".join(j)).split("@")
#                     value.append(s[1])
#     else:
#         m.append(k)
#         s = (" ".join(m)).split("@")
#         try:
#             value.append(s[1])
#         except:
#             pass
#     print(i,list(set(value)))

In [165]:
import pandas as pd
df_1 = pd.read_csv("md_97y_phrase_1.csv")
df_1.shape

(2493, 7)

In [166]:
df_1.head()

,md5,subject,from_domain,to_domain,begin_email_date,end_email_date,Domains
0,['a44c407f804960d920486ecd0cf45d8c'],['Re: RE: RE: RE: RE: Re: 1% Vendor Fund Initi...,['guangwei-china.com'],['cantire.com'],[' October 10 2019 '],[' September 4 2020 '],guangwei-china.com
1,['cada51b7c31e0bc056dd7b039643daf1'],['Re: RE: RE: RE: Re: 1% Vendor Fund Initiativ...,['guangwei-china.com'],['cantire.com'],[' October 10 2019 '],[' December 18 2019 '],guangwei-china.com
2,['1c0bc1b741d35cacf52f25b1bd1e6d23'],['Re: Re: RE: RE: RE: Re: 1% Vendor Fund Initi...,['guangwei-china.com'],['cantire.com'],[' October 10 2019 '],[' December 18 2019 '],guangwei-china.com
3,['4b24e4b46998bf61e3f6720b38c7d1f8'],['RE: RE: RE: RE: Re: 1% Vendor Fund Initiativ...,['cantire.com'],['guangwei-china.com'],[' October 10 2019 '],[' September 4 2020 '],guangwei-china.com
4,['339d20dc5ab218edcff48a75de4ae7e7'],['RE: Re: RE: RE: RE: Re: 1% Vendor Fund Initi...,['cantire.com'],['guangwei-china.com'],[' October 10 2019 '],[' September 11 2020 '],guangwei-china.com


In [27]:
df_2 = pd.read_csv("vendor_data2 (1).csv")
df_2.shape

(12785, 5)

In [64]:
df_3 = df_2.loc[:,["Domains","CME_ID"]]
df_3

,Domains,CME_ID
0,sharkninja.com,4555
1,sharkninja.com,8852
2,sharkninja.com,9680
3,sharkninja.com,6479
4,sharkninja.com,G6NY
...,...,...
12780,torys.com,NaN
12781,classycaps.com,G3RJ
12782,neopost.com,NaN
12783,NaN,NaN


In [123]:
for i in df_1["Domains"]:
    print(i)
    break
    val = df_3.loc[df_3['Domains'] == i, 'CME_ID']
    print(val)

guangwei-china.com


In [167]:
import pandas as pd
new = pd.DataFrame(columns=["CME-ID"])

In [168]:
new

,CME-ID


In [171]:
import numpy as np
u = 0
for i in df_1["Domains"]:
    lis = []
    val = df_3.loc[df_3["Domains"] == i, "CME_ID"]
    val = val[~val[:].isna()]
    for p in val:
        lis.append(p)
    lis_val = ""
    for i in lis:
        lis_val += i+","
    new.loc[u] = lis_val
    u+=1

In [172]:
new

,CME-ID
0,"1747,"
1,"1747,"
2,"1747,"
3,"1747,"
4,"1747,"
...,...
2488,"G694,"
2489,"G694,"
2490,"G694,"
2491,"G694,"


In [173]:
df_ = df_1.join(new)

In [180]:
len([val for val in df_["CME-ID"] if len(val)==0])

901

In [174]:
df_.to_csv("md_97y_fin_1.csv")

In [108]:
len(df_3["Domains"].unique())
df_3[df_3["Domains"][:] == "cantire.com"]

,Domains,CME_ID
238,cantire.com,NaN
1883,cantire.com,NaN
2363,cantire.com,NaN
2923,cantire.com,NaN
5779,cantire.com,NaN
6556,cantire.com,G694
6624,cantire.com,NaN
7441,cantire.com,NaN
7773,cantire.com,NaN
8079,cantire.com,NaN


In [66]:
df_ = pd.merge(df_1,df_3, on="Domains", how='left')
df_.shape

(27301, 8)

In [67]:
df_

,md5,subject,from_domain,to_domain,begin_email_date,end_email_date,Domains,CME_ID
0,['a44c407f804960d920486ecd0cf45d8c'],['Re: RE: RE: RE: RE: Re: 1% Vendor Fund Initi...,['guangwei-china.com'],['cantire.com'],[' October 10 2019 '],[' September 4 2020 '],guangwei-china.com,1747
1,['cada51b7c31e0bc056dd7b039643daf1'],['Re: RE: RE: RE: Re: 1% Vendor Fund Initiativ...,['guangwei-china.com'],['cantire.com'],[' October 10 2019 '],[' December 18 2019 '],guangwei-china.com,1747
2,['1c0bc1b741d35cacf52f25b1bd1e6d23'],['Re: Re: RE: RE: RE: Re: 1% Vendor Fund Initi...,['guangwei-china.com'],['cantire.com'],[' October 10 2019 '],[' December 18 2019 '],guangwei-china.com,1747
3,['4b24e4b46998bf61e3f6720b38c7d1f8'],['RE: RE: RE: RE: Re: 1% Vendor Fund Initiativ...,['cantire.com'],['guangwei-china.com'],[' October 10 2019 '],[' September 4 2020 '],guangwei-china.com,1747
4,['339d20dc5ab218edcff48a75de4ae7e7'],['RE: Re: RE: RE: RE: Re: 1% Vendor Fund Initi...,['cantire.com'],['guangwei-china.com'],[' October 10 2019 '],[' September 11 2020 '],guangwei-china.com,1747
...,...,...,...,...,...,...,...,...
27296,NaN,NaN,NaN,NaN,NaN,NaN,versapet.com,G515
27297,NaN,NaN,NaN,NaN,NaN,NaN,foremostgroups.com,G46F
27298,NaN,NaN,NaN,NaN,NaN,NaN,kidsvip.ca,G6F6
27299,NaN,NaN,NaN,NaN,NaN,NaN,vicwestbrands.com,NaN


In [48]:
df_.to_csv("md_97y_fin.csv")

In [173]:
cd zz

/mnt/cantire-analytics/datarepo/zz


In [249]:
df = pd.read_excel("entityExtractionCD_290.xlsx")
df.head(1)
df.shape

(22543, 17)

In [250]:
md5s = df["md5"].unique()
len(md5s)

234

In [251]:
md5s[0]

'595c0a4495d5a7cda7e6be80349050b3'

In [252]:
# import csv
# header = ["md5","from","to","subject","email_date"]
# with open("entity_ext_290.csv","a") as f:
#             csvwriter = csv.writer(f)
#             csvwriter.writerow(header)

In [366]:
from tqdm import tqdm
i=0
lis = []
for name in tqdm(md5s):
    i+=1
    body = {
        "query":{
            "bool":{
                "must":[{
                    "match_phrase":{
                        "Attachment.MD5":name
                    }
                },
                {
                    "match_phrase_prefix":{
                        "fileType":"Email"
                    }
                }]
            }
        }
    }
    res = es.search(index=["cantire-2021-*","cantire-2022-*"],body = body)
    frm = []
    to = []
    sub = []
    date = []
    try:
        f = res["hits"]["hits"][0]["_source"]["metaData"]["from"]
        if type(f)!=str:
            frm.extend(f)
        else:
            frm.append(f)
    except:
        pass
    try:
        t = res["hits"]["hits"][0]["_source"]["metaData"]["to"]
        if type(t)!=str:
            to.extend(t)
        else:
            to.append(t)
    except:
        pass
    try:
        s = res["hits"]["hits"][0]["_source"]["metaData"]["subject"]
        if type(s)!=str:
            sub.extend(s)
        else:
            sub.append(s)
    except:
        pass
    try:
        d = res["hits"]["hits"][0]["_source"]["metaData"]["date"]
        if type(d)!=str:
            date.extend(d)
        else:
            date.append(d)
    except:
        pass
    lis.extend([[name,frm,to,sub,date]])
    print(lis)
    break
#     if i==100:
#         with open("entity_ext_290.csv","a") as file:
#             csvwriter = csv.writer(file)
#             csvwriter.writerows(lis)
#         lis=[]
#         i=0
# if i!=0:
#     with open("entity_ext_290.csv","a") as file:
#             csvwriter = csv.writer(file)
#             csvwriter.writerows(lis)

  0%|          | 0/234 [00:02<?, ?it/s]

[['595c0a4495d5a7cda7e6be80349050b3', ['Melissa.McCreary@cantire.com'], ['Madison.Merritt@cantire.com'], ['Goodbaby Canada Cost Down D333-336-337'], ['2021/10/15 15:52:06']]]


In [254]:
import pandas as pd
df1 = pd.read_excel("entityExtractionCD_290.xlsx")
df1.head(1)

,sku,product_description,cost_per_unit,vendor_name,vendor_number,date,purchase_agreement_date,cbm_number,cbm_name,contract_number,promotional_deal,signatures,format,source,filename,md5,program_name
0,0463980 EVNFL SYMPHONY,20,20,Goodbaby Canada Inc,7516,2021-10-15 00:00:00,NaN,139,Elena Branch,21-333-336-337-137-7516-POST,NaN,0,old,email,21-333-336-337-139-7516-POST.pdf,595c0a4495d5a7cda7e6be80349050b3,NaN


In [255]:
df2 = pd.read_csv("entity_ext_290.csv")
len([val for val in df2["email_date"] if len(eval(val))==0])

32

In [256]:
k = []
for i in range(len(df["email_date"])):
    if len(df["email_date"][i]) <= 4:
        k.append(i)

In [257]:
len(k)

32

In [258]:
import csv
header = ["md5"]
with open("290_missing_md5_lis.csv","a") as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(header)

In [259]:
for i in k:
    lis = []
    lis.append(df2["md5"][i])
    with open("290_missing_md5_lis.csv","a") as f:
                csvwriter = csv.writer(f)
                csvwriter.writerow(lis)

In [261]:
df2.head(1)

,md5,from,to,subject,email_date
0,595c0a4495d5a7cda7e6be80349050b3,['Melissa.McCreary@cantire.com'],['Madison.Merritt@cantire.com'],['Goodbaby Canada Cost Down D333-336-337'],['2021/10/15 15:52:06']


In [263]:
df2["email_date"][0]

"['2021/10/15 15:52:06']"

In [239]:
df2["filename"][36]

'0804e9277f9eb766cf2158ebcc738565'

In [265]:
df = pd.merge(df1, df2, on='md5', how='left')
df.shape

(22543, 21)

In [266]:
df.to_csv("entitiy_extr_290_fin.csv")

In [192]:
df = pd.read_csv("oa-sep-oct-fin.csv")
df.head()

,Unnamed: 0,vendor_number,vendor_name,cbm_number,cbm_name,contract_number,contract_date,amount,program_name,filename,md5,from,to,subject,email_date
0,0,9680,EURO-PRO CORPORATION,30.0,NaN,NaN,2022-09-12 00:00:00,200.06,Vendor Funds to Support Webshop Orders for DWO...,Shark_9680_-_Margin_O&A_Q1_Q2_-_Webshop_Orders...,dabd09db079b69c5e71cff3fd6ca995c,['Meg.Casbourn@cantire.com'],"['John.Hooke@cantire.com', 'REddy@sharkninja.c...",['[EXTERNAL] - RE: SHARK ALM Activity Tracking...,['2022/09/21 13:27:13']
1,1,3290,RUST-OLEUM CORP,NaN,NaN,NaN,NaN,Per Unit,2022 Q1 Cost Downs,22-0304-3290-163.pdf,965aebacf070da9ef75b1c45e0623023,[],[],[],[]
2,2,NaN,NaN,58.0,NaN,NaN,NaN,Per Unit ($),2022 Q4 Cost Downs,22-0349-4109-058.pdf,0e0f9d79ac55cf5cdd790b69eaf9a947,"['""Houle', 'houle.t@pg.com']",['Andrew.Holder@cantire.com'],['[EXTERNAL] - RE: Deals 0349/350/351 Updated ...,['2022/09/14 19:55:02']
3,3,NaN,NaN,NaN,NaN,NaN,NaN,See Schedule A attached hereto,Deal 329/2022,Deal_329_Cost_Down_OA_Agreement_-_v9344.pdf,feacec6cb818338171c53fb5af32483e,[],[],[],[]
4,4,4520,MTD PRODUCTS LTD.,90.0,NaN,NaN,August 29th 2022,14569,Non Advertising O&A,OverAbove_2022_MTDZT.pdf,48e1cf410ba92719d34f1b8e3f82ce03,['Karl.Solis@cantire.com'],['Vendor.Support@cantire.com'],['[EXTERNAL] - FW: O&A for ZT Tractors - V# 21...,['2022/09/26 15:59:52']


In [199]:
len(df["email_date"].isna())

1774

In [205]:
len([val for val in df["email_date"] if len(eval(val))==0])

807